In [2]:
import numpy as np
from matplotlib import pyplot as plt 

In [7]:
arr =np.array(
    [[[1,-1],[1,-1],[-1,1]],
     [[-1,-1],[-1,1],[1,1]],
     [[1,1],[-1,-1],[-1,1]]])

In [8]:
arr

array([[[1, 1],
        [1, 0],
        [0, 1]],

       [[0, 0],
        [0, 1],
        [1, 1]],

       [[1, 1],
        [0, 0],
        [0, 1]]])

In [14]:
#This corresponds to pic in book
arr = [[[-1,1],[-1,1],[1,-1],[-1,1]],
       [[-1,-1],[-1,-1],[-1,1],[1,-1]],
       [[-1,1],[-1,1],[1,1],[-1,1]],
       [[-1,1],[-1,1],[1,1],[-1,1]]]
arr = np.array(arr)

In [15]:
arr 

array([[[-1,  1],
        [-1,  1],
        [ 1, -1],
        [-1,  1]],

       [[-1, -1],
        [-1, -1],
        [-1,  1],
        [ 1, -1]],

       [[-1,  1],
        [-1,  1],
        [ 1,  1],
        [-1,  1]],

       [[-1,  1],
        [-1,  1],
        [ 1,  1],
        [-1,  1]]])

In [ ]:
def initialise_state(pts):
    ###Create a valid state
    
    #Just 
    return arr

In [ ]:
def visualise_2d_model(arr):
    ###
    
    
    print(model)

In [ ]:
def current_config_is_valid(arr):
    #This won't hurt
    #Might be computationally intensive
    #But shouldn't be used much
    
    
    
    return boo

In [ ]:
def long_loop(arr):
    #Modify arr
    return new_state